
# Processamento Textual: Teoria e Prática com Word2Vec, Clusterização e LSTMs

Este notebook foi desenvolvido para uma aula completa com explicações teóricas e práticas. Ele aborda processamento textual 
em profundidade e inclui análises detalhadas de cada etapa.

### Estrutura do Notebook
1. **Introdução ao Processamento Textual**:
   - Contexto geral sobre processamento de linguagem natural (NLP).
   - Objetivos do notebook.
2. **Embeddings Word2Vec**:
   - O que são embeddings e como Word2Vec funciona.
   - Geração de embeddings a partir de um corpus sintético.
3. **Pré-processamento de Texto**:
   - Técnicas de limpeza de texto, stemming e lematização.
   - Explicação detalhada do dataset `movie_reviews` do NLTK.
4. **Clusterização de Palavras**:
   - Explicação de K-Means e como agrupar embeddings.
   - Visualização dos clusters e da similaridade entre palavras.
5. **Redes LSTM**:
   - O que são redes LSTM e como são usadas em NLP.
   - Treinamento e análise detalhada de uma LSTM para modelagem de sequência.

Cada seção inclui uma explicação teórica antes da execução do código para facilitar o entendimento.



## Embeddings Word2Vec

Os embeddings são representações vetoriais das palavras em um espaço contínuo, permitindo que palavras semanticamente semelhantes fiquem próximas nesse espaço. 
O Word2Vec é um dos modelos mais populares para criar embeddings e utiliza dois métodos principais:

- **Skip-Gram**: Prediz o contexto de uma palavra a partir da palavra alvo.
- **CBOW (Continuous Bag of Words)**: Prediz a palavra alvo a partir de seu contexto.

Nesta seção, vamos criar embeddings utilizando um corpus sintético simples. O objetivo é demonstrar como treinar um modelo Word2Vec e analisar as representações geradas.


In [ ]:

from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk
import numpy as np

nltk.download("punkt")

# Corpus sintético simples
corpus = [
    "rei homem mulher rainha",
    "rei rainha trono castelo",
    "homem mulher menino menina",
    "rei castelo cavaleiro rainha",
]

# Tokenização
tokenized_corpus = [word_tokenize(sentence) for sentence in corpus]

# Treinamento do modelo Word2Vec
model = Word2Vec(sentences=tokenized_corpus, vector_size=50, window=3, min_count=1, workers=4)
print("Modelo Word2Vec treinado!")

# Análise de embeddings
print("Vetor para 'rei':", model.wv["rei"][:5])
print("Palavras semelhantes a 'rei':", model.wv.most_similar("rei", topn=3))



## Pré-processamento de Texto

O pré-processamento é uma etapa essencial em NLP. Ele prepara os dados textuais para análise e modelos de aprendizado. 
As etapas comuns incluem:

1. **Tokenização**: Divisão do texto em palavras ou frases.
2. **Remoção de Stopwords**: Palavras comuns como "o", "de", "a" que não contribuem para a análise.
3. **Stemming**: Redução das palavras às suas raízes (ex.: "jogando" → "jog").
4. **Lematização**: Redução das palavras às suas formas base (ex.: "jogando" → "jogar").

### Dataset: `movie_reviews` (NLTK)
- Este dataset contém resenhas de filmes categorizadas como positivas ou negativas.
- É frequentemente usado para análise de sentimentos.


In [ ]:

from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import movie_reviews, stopwords

nltk.download("wordnet")
nltk.download("stopwords")
nltk.download("movie_reviews")

# Carregar o dataset
texts = [movie_reviews.raw(fileid) for fileid in movie_reviews.fileids()[:3]]

# Tokenização e remoção de stopwords
stop_words = set(stopwords.words("english"))
filtered_texts = []
for text in texts:
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    filtered_texts.append(tokens)

# Stemming e lematização
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
stemmed = [[stemmer.stem(word) for word in tokens] for tokens in filtered_texts]
lemmatized = [[lemmatizer.lemmatize(word) for word in tokens] for tokens in filtered_texts]

print("Exemplo de texto processado (stemming):", stemmed[0][:10])
print("Exemplo de texto processado (lematização):", lemmatized[0][:10])



## Clusterização de Palavras

A clusterização é uma técnica de aprendizado não supervisionado para agrupar dados similares. 
Usaremos o algoritmo **K-Means** para agrupar palavras baseadas em seus embeddings.

Além disso, visualizaremos:
- Um gráfico de dispersão dos clusters.
- Uma matriz de similaridade entre palavras usando um heatmap.


In [ ]:

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

# Clusterização com K-Means
words = list(model.wv.index_to_key)
vectors = np.array([model.wv[word] for word in words])

kmeans = KMeans(n_clusters=2, random_state=42)
labels = kmeans.fit_predict(vectors)

# Visualização dos clusters
plt.figure(figsize=(10, 7))
for i, word in enumerate(words):
    plt.scatter(vectors[i, 0], vectors[i, 1], c=f"C{labels[i]}")
    plt.text(vectors[i, 0] + 0.02, vectors[i, 1] + 0.02, word, fontsize=9)
plt.title("Clusters de Palavras")
plt.show()

# Matriz de similaridade
similarity_matrix = np.dot(vectors, vectors.T)
plt.figure(figsize=(10, 8))
sns.heatmap(similarity_matrix, xticklabels=words, yticklabels=words, cmap="coolwarm")
plt.title("Matriz de Similaridade entre Palavras")
plt.show()


#### O que o gráfico representa?
Eixos (X e Y):

Estes eixos correspondem a uma projeção bidimensional dos embeddings, que originalmente possuem 50 dimensões (como definido no treinamento do Word2Vec).
A redução para 2 dimensões foi feita para possibilitar a visualização gráfica, provavelmente utilizando um método como PCA ou TSNE.
Pontos:

Cada ponto no gráfico representa uma palavra do corpus.
As posições dos pontos refletem a similaridade semântica entre as palavras no espaço vetorial. Palavras próximas têm significados ou contextos semelhantes.
Cores:

As cores indicam os clusters formados pelo algoritmo K-Means.
As palavras dentro do mesmo cluster compartilham características similares no espaço vetorial.

#### Análise detalhada
###### Cluster Azul:
- Contém palavras como "rei", "castelo", e "trono".
- Essas palavras são semanticamente relacionadas, pois todas estão associadas ao contexto de realeza e ambientes monárquicos.
- Isso mostra que o modelo Word2Vec capturou corretamente as relações semânticas do corpus.
###### Cluster Laranja:
- Inclui palavras como "homem", "mulher", "menino", e "menina".
- Essas palavras estão relacionadas a gêneros e faixas etárias, formando um agrupamento distinto dos termos relacionados à realeza.

###### Posições específicas:
- "Rei" e "trono": Estão muito próximos, indicando que o modelo aprendeu uma relação semântica forte entre eles (o que faz sentido, pois "rei" geralmente está associado a um trono no corpus).
- "Mulher": Está mais distante de "rei" e "trono", pois pertence ao cluster laranja, que trata de conceitos diferentes (gênero e idade).
- "Menino" e "menina": Estão próximos entre si, refletindo a similaridade semântica entre os termos.

##### Gráfico 2: Matriz de Similaridade entre Palavras
Este gráfico é um heatmap que mostra a similaridade entre palavras no espaço vetorial. Cada célula representa o produto escalar entre os vetores de duas palavras.

O que foi feito:
- Matriz de Similaridade: A similaridade foi calculada usando o produto escalar entre os vetores de cada par de palavras.
- Visualização com Heatmap: A matriz foi plotada como um mapa de calor para facilitar a interpretação.
Interpretação do Heatmap:
- Cores mais claras/vermelhas: Indicam maior similaridade entre palavras (produto escalar maior).
- Cores mais escuras/azuladas: Indicam menor similaridade entre palavras (produto escalar menor ou negativo).

Por exemplo:
- A similaridade entre "rei" e "rainha" é alta (vermelho), indicando que essas palavras compartilham muitos atributos semânticos no corpus.
- Palavras como "rei" e "cavaleiro" também apresentam uma certa similaridade, mas menos intensa do que a entre "rei" e "rainha".
- Palavras como "trono" e "menino" possuem baixa similaridade, refletindo seu significado distante no corpus.



### Por que utilizar RNNs em Análise de Sentimentos?
As Redes Neurais Recorrentes (RNNs) são amplamente utilizadas em tarefas de Processamento de Linguagem Natural (NLP), como análise de sentimentos, devido à sua capacidade de processar dados sequenciais. Aqui estão as razões principais para o uso de RNNs nessa tarefa:

#### Natureza Sequencial do Texto
O texto é uma sequência de palavras, onde o significado de uma palavra muitas vezes depende do contexto em que está inserida.
RNNs são projetadas para capturar relações e padrões em dados sequenciais, analisando uma palavra ou caractere considerando as informações anteriores.

#### Memória de Longo Alcance
RNNs têm uma memória interna que armazena informações sobre o que já foi processado.
Isso permite que a rede compreenda dependências de longo prazo no texto, o que é essencial para capturar nuances de sentimento em frases mais longas.




##### Capacidade de Modelar Contexto
As palavras em um texto podem ter diferentes significados dependendo do contexto.
RNNs conseguem capturar essas diferenças, entendendo o sentimento implícito nas relações entre as palavras.

##### Eficiência em Capturar Padrões Temporais
- RNNs processam sequências de forma recorrente, atualizando sua memória a cada etapa.
- Isso permite que a rede detecte padrões temporais no texto, como negações ou contrastes.


## Redes LSTM

As redes LSTM são uma variante das redes neurais recorrentes, projetadas para capturar dependências de longo prazo em dados sequenciais. 
Nesta seção, treinaremos uma LSTM para prever a próxima palavra em sequências textuais.

### Etapas:
1. Criar um corpus sintético simples.
2. Gerar sequências para treinamento.
3. Treinar a LSTM e visualizar a evolução da perda.
4. Usar o modelo para gerar texto.


In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Corpus sintético
corpus = [
    "rei homem mulher rainha",
    "rainha trono castelo cavaleiro",
    "homem menino menina",
    "castelo cavaleiro rainha",
]

# Tokenização
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1
input_sequences = []

# Criar sequências de entrada
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Padding
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding="pre")

# Dividir entradas e saídas
xs, ys = input_sequences[:,:-1], input_sequences[:,-1]

# Modelo LSTM
model = Sequential()
model.add(Embedding(total_words, 10, input_length=max_sequence_len-1))
model.add(LSTM(100))
model.add(Dense(total_words, activation="softmax"))
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Treinamento com verbose
history = model.fit(xs, ys, epochs=50, verbose=1, batch_size=8)

# Visualização da perda
plt.plot(history.history["loss"], label="Loss")
plt.title("Evolução da Perda Durante o Treinamento")
plt.xlabel("Épocas")
plt.ylabel("Perda")
plt.legend()
plt.show()

# Geração de texto
seed_text = "rei homem"
next_words = 3

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding="pre")
    predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)
    output_word = tokenizer.index_word[predicted[0]]
    seed_text += " " + output_word

print("Texto gerado:", seed_text)


##### Gráfico da Perda Durante o Treinamento
O que é o gráfico de perda?
O gráfico mostra como a função de perda do modelo evolui ao longo das épocas de treinamento.
A perda é uma métrica que mede o quão bem o modelo está aprendendo a prever a saída correta (neste caso, a próxima palavra na sequência).
No início do treinamento, o modelo começa com alta perda porque ainda não "aprendeu" padrões no corpus.
À medida que o treinamento avança, a perda diminui, indicando que o modelo está melhorando.
O que o gráfico mostra?
Início com alta perda (2.3):

No início, o modelo está apenas começando o treinamento. Ele não conhece o padrão de sequência no corpus.
Isso resulta em previsões aleatórias e alta perda.
Redução progressiva da perda:

A curva desce de forma constante, mostrando que o modelo está aprendendo os padrões no corpus.
A redução não é linear, pois em algumas épocas o progresso é menor (indicando que o modelo está refinando o aprendizado).
Perda final (~1.7):

O modelo estabiliza em torno de 1.7 ao final do treinamento.
Isso indica que o modelo aprendeu a gerar sequências com base no corpus, mas ainda pode haver limitações devido ao tamanho do corpus ou à complexidade do modelo.


#### Análise dos Resultados
A figura apresenta dois elementos principais: o gráfico de perda (loss) durante o treinamento do modelo LSTM e o texto gerado pelo modelo. Vou explicar cada um deles detalhadamente.

1. Gráfico da Perda Durante o Treinamento
O que é o gráfico de perda?
O gráfico mostra como a função de perda do modelo evolui ao longo das épocas de treinamento.
A perda é uma métrica que mede o quão bem o modelo está aprendendo a prever a saída correta (neste caso, a próxima palavra na sequência).
No início do treinamento, o modelo começa com alta perda porque ainda não "aprendeu" padrões no corpus.
À medida que o treinamento avança, a perda diminui, indicando que o modelo está melhorando.
O que o gráfico mostra?
Início com alta perda (2.3):

No início, o modelo está apenas começando o treinamento. Ele não conhece o padrão de sequência no corpus.
Isso resulta em previsões aleatórias e alta perda.
Redução progressiva da perda:

A curva desce de forma constante, mostrando que o modelo está aprendendo os padrões no corpus.
A redução não é linear, pois em algumas épocas o progresso é menor (indicando que o modelo está refinando o aprendizado).
Perda final (~1.7):

O modelo estabiliza em torno de 1.7 ao final do treinamento.
Isso indica que o modelo aprendeu a gerar sequências com base no corpus, mas ainda pode haver limitações devido ao tamanho do corpus ou à complexidade do modelo.
2. Texto Gerado pelo Modelo
Texto: "rei homem rainha rainha rainha"
Como o texto foi gerado?
O modelo LSTM foi treinado para prever a próxima palavra em uma sequência.
O texto gerado começa com a semente inicial ("rei homem") e, a cada passo, o modelo prevê a próxima palavra com base nas palavras anteriores.
Por que temos "rainha rainha rainha"?
Tamanho limitado do corpus:

O corpus usado para o treinamento é pequeno, com apenas algumas frases curtas.
Isso limita a capacidade do modelo de capturar variações mais amplas no padrão de sequência.
Tendência de repetição:

"Rainha" é uma palavra comum no corpus e ocorre frequentemente em relação a "rei" e "homem".
O modelo pode ter aprendido uma forte associação entre essas palavras, resultando em repetições.
Memória limitada da LSTM:

Modelos LSTM têm memória limitada, especialmente quando o tamanho da rede é pequeno.
Isso pode levar à geração de padrões repetitivos em vez de variações criativas.